In [1]:
from transformers import TFAutoModel 

model_path = "claudios/cubert-20210711-Java-1024"
model = TFAutoModel.from_pretrained(model_path)

/home/sigmazer/GP/cubert/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-01 17:59:14.662696: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738425554.715741  706573 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738425554.730105  706573 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-01 17:59:14.823736: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical o

In [3]:
import pandas as pd

train_df = pd.read_csv("train_beautified.csv")
test_df = pd.read_csv("test_beautified.csv")

In [4]:
from cubert.full_cubert_tokenizer import FullCuBertTokenizer
from cubert import java_tokenizer

fullTokenizer = FullCuBertTokenizer(java_tokenizer.JavaTokenizer, "20210711_Java_github_java_deduplicated_vocabulary.txt")

In [ ]:
import torch
import numpy as np
import tensorflow as tf
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm  import tqdm

split_lines = [line.split("\n") for line in train_df["code"]]

train_df_new = pd.DataFrame(columns=["embedding", "godclass"])

def process_sample(sample):
    sample_final = np.zeros(1024) 

    for line in sample:
        inputs = fullTokenizer.convert_tokens_to_ids(fullTokenizer.tokenize(line))
        if len(inputs) == 0:
            continue  

        inputs = tf.convert_to_tensor([inputs])  
        outputs = model(inputs)  

        sample_final += outputs[0][0][0].numpy()
    print("one done :\"")
    return sample_final

results = []
with ProcessPoolExecutor() as executer:
    futures = {executer.submit(process_sample, sample): i for i, sample in enumerate(split_lines)}

    for future in tqdm(as_completed(futures), total=len(split_lines), desc="Processing Embeddings"):
        results.append(future.result())


train_df_new["embedding"] = results
train_df_new["godclass"] = train_df["godclass"]

train_df_new.to_csv("train_embeddings.csv", index=False)

print("Processing completed!")



In [ ]:
import torch
import numpy as np
from tqdm import tqdm
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

split_lines = [line.split('\n') for line in train_df['code']]

train_df_new = pd.DataFrame(columns = ['embedding', 'godclass'])

for i, sample in enumerate(split_lines):
    sample_final = np.zeros(1024)
    for line in tqdm(sample):
        with torch.no_grad():
            inputs = fullTokenizer.convert_tokens_to_ids(fullTokenizer.tokenize(line))
            if len(inputs) == 0:
                continue
            inputs = tf.convert_to_tensor([inputs])
            outputs = model(inputs)
            sample_final += outputs[0][0][0].numpy()

    train_df_new = train_df_new.append({'embedding': sample_final, 'godclass': train_df['godclass'][i]}, ignore_index=True)